In [5]:
import csv
from bs4 import BeautifulSoup
from selenium import webdriver


def get_url(search_term):
    template = 'https://www.amazon.in/s?k={}&ref=nb_sb_noss'
    search_term = search_term.replace(' ','+')
    url = template.format(search_term)
    url = url + '&page{}'
    return url


def extraction(item):
    antag = item.h2.a
    description = antag.text.strip()
    url = 'https://www.amazon.com' + antag.get('href')
    
    # price
    try:
        price_parent = item.find('span','a-price')
        price = price_parent.find('span','a-offscreen').text
        
    except AttributeError:
        return 

    
    try:
        # ratings
        rating = item.i.text

        # review_count
        review_count = item.find('span', {'class': 'a-size-base s-underline-text'}).text
    
    except AttributeError:
        rating = ''
        review_count = ''
        
        
    
    # storing all in a tuple
    result = (description , price , rating , review_count , url)
    
    return result
    

    
def base(search_term):        
    driver = webdriver.Chrome()   
    records = []   
    url = get_url(search_term)
    
    for page in range(1,21):
        driver.get(url.format(page))
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        results = soup.find_all('div',{'data-component-type': 's-search-result'})
        
        for item in results:
            record = extraction(item)
            if record:
                records.append(record)
                
    driver.close()

    with open('results.csv','w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(['Description' , 'Price' , 'Rating' , 'ReviewCount' , 'Url'])
        writer.writerows(records)
        
      
base('bags')

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=109.0.5414.120)
Stacktrace:
Backtrace:
	(No symbol) [0x01086643]
	(No symbol) [0x0101BE21]
	(No symbol) [0x00F1DA9D]
	(No symbol) [0x00EFEF6A]
	(No symbol) [0x00F73AAB]
	(No symbol) [0x00F861B6]
	(No symbol) [0x00F6FB76]
	(No symbol) [0x00F449C1]
	(No symbol) [0x00F45E5D]
	GetHandleVerifier [0x012FA142+2497106]
	GetHandleVerifier [0x013285D3+2686691]
	GetHandleVerifier [0x0132BB9C+2700460]
	GetHandleVerifier [0x01133B10+635936]
	(No symbol) [0x01024A1F]
	(No symbol) [0x0102A418]
	(No symbol) [0x0102A505]
	(No symbol) [0x0103508B]
	BaseThreadInitThunk [0x76CF00F9+25]
	RtlGetAppContainerNamedObjectPath [0x77657BBE+286]
	RtlGetAppContainerNamedObjectPath [0x77657B8E+238]
